# Spark SQL Operations

Data for this notebook is from the **yelp-data-challenge**: https://www.yelp.com/dataset_challenge/

## Checking Spark Version and Configuration

Once you run the chunk above, the notebook will create a few Spark configuration variables for you. The primary variable is the SparkSession variable defined as spark.

In [ ]:
println(spark.version)
println(sc.version)
println(sc.appName)

## Reading Data into Spark DataFrames

To begin our analysis, we will read in some data into Spark DataFrames. The data we will be analyzing is the yelp data we have saved in HDFS under `/yelp/yelp-data/`. Let's read in the business dataset first. Here we'll use the built-in json `DataFrameReader` from the `spark` Spark Session object.

In [ ]:
val business = spark.read.json("/yelp/data/yelp_academic_dataset_business.json")

In [ ]:
business.show()

In [ ]:
business.printSchema()

In [ ]:
val columnNames = Seq("business_id", "name", "city", "stars", "state",
                      "categories", "address", "categories", "review_count")

In [ ]:
val biz = business.select(columnNames.map(c => col(c)): _*)


In [ ]:
biz.show()

In [ ]:
biz.count()

## Running SQL Queries Against DataFrames

The `sql` function within a SparkSession enables applications to run SQL queries programmatically and returns the result as a DataFrame. In HDInsight Jupyter notebooks, there are a few additional [parameters](https://docs.microsoft.com/en-us/azure/hdinsight/hdinsight-apache-spark-jupyter-notebook-kernels#parameters-supported-with-the-sql-magic) you can use to manipulate the output from SQL queries.

In order to run queries against a Spark DataFrame, you need to first create a temporary view for Spark SQL.

In [ ]:
business.createOrReplaceTempView("business")

In [ ]:
spark.sql("show tables").show()

In [ ]:
%%sql
SHOW TABLES

In [ ]:
%%sql 
SELECT state, COUNT(*) as cnt FROM business GROUP BY state ORDER BY cnt DESC

In [ ]:
%%sql 
SELECT city, COUNT(*) as cnt FROM business GROUP BY city ORDER BY cnt DESC

## Examining Reviews Data

In [ ]:
val reviews = spark.read.json("/yelp/data/yelp_academic_dataset_review.json")

In [ ]:
reviews.printSchema()

In [ ]:
reviews.groupBy("stars").count().explain(true)

In [ ]:
reviews.groupBy("stars").count().show()

In [ ]:
reviews.groupBy("stars", "funny").count().sort(desc("count")).show()

## Joins

![](http://i.imgur.com/x501fi0.gif)

In [ ]:
val biz_reviews = business.join(reviews, 
                                business.col("business_id") === reviews.col("business_id"), 
                                "left_outer")

In [ ]:
biz_reviews.explain(true)

In [ ]:
biz_reviews.count()

In [ ]:
biz_reviews.show()